In [16]:
import pandas as pd

def get_top_usluga_url(url:str) -> str:
    return url[:url.rfind('/')]

def get_relative_url(url:str) -> str:
    return url.replace('https://ddcar.ru', '')

def get_end_usluga_url(url:str) -> str:
    return url[url.rfind('/'):]


usluga = 'Замена топливного фильтра'
usluga_url = '/diagnostika/zamena-toplivnogo-filtra'    

data_read = pd.read_csv('marka_model.csv', encoding = 'utf-8-sig', sep=';')
# добавляем столбец с услугой к данным
data_read['Услуга'] = usluga
db_all_page = pd.read_csv('all_export_20200225-095421.csv', encoding = 'cp1251', sep=';')
print(db_all_page['Основной текст'])
db_all_page = db_all_page.iloc[:, :6]

#находим родительский раздел, в котором лежит услуга
top_usluga_url = get_top_usluga_url(usluga_url)

#ищем название родительской услуги
top_usluga_name = False

data_filter_url = db_all_page[db_all_page['_URL_'].str.endswith(f'{top_usluga_url}')]
if len(data_filter_url):
    top_usluga_name = data_filter_url['Услуга'].reset_index(drop=True)[0]  
else: print(f'Нет родительской услуги: {top_usluga_url}')
#если название родительской услуги есть, то ищем раздел в выгрузке с пересечением марки и модели
if top_usluga_name:
    for idx, data in data_read.iterrows():
        data_filter_marka = db_all_page[db_all_page['Марка'] == data.Марка]
        if pd.notna(data.Модель):
            data_filter_model = data_filter_marka[data_filter_marka['Модель'] == data.Модель]
        else:
            data_filter_model = data_filter_marka[pd.isna(data_filter_marka['Модель'])]
        data_filter_model_url = data_filter_model[data_filter_model['Услуга'] == top_usluga_name]
        if len(data_filter_model_url):
            url_change = data_filter_model_url._URL_ + get_end_usluga_url(usluga_url)
            data_read.loc[idx, ['URL']] = url_change.item()
            print(url_change)
        else:
            print(f'Не найдена услуга {top_usluga_name} для \n{data}')
            data_read.loc[idx, ['URL']] = get_relative_url(data.URL + usluga_url)


print(data_read['URL'])
data_read.to_csv('file_import_with_chek_for_top_usluga.csv', sep= ";", encoding = "utf-8-sig")

#выбираем из общей выгрузки те, что совпадают по маркам+моделям
#db_all_page = db_all_page.loc[db_all_page['Марка'] == data['Марка'][1]]
#db_all_page = db_all_page.loc[db_all_page['Модель'] == data['Модель'][1]]

#db_all_page = db_all_page.loc[db_all_page['_URL_'] == data['Модель'][1]]
#data = data.loc[data['Марка'] == data['Марка'][1]]
#data = data.loc[db_all_page['Модель'] == data['Модель'][1]]



0        <p>На смену старым карбюраторным системам пода...
1        <p>Бензонасос откачивает топливо из бака и под...
2        <p>Дизельные форсунки &ndash; это небольшие на...
3        <p>Инжекторная система подачи топлива отличает...
4        <p>Автомобиль, на котором установлена неисправ...
                               ...                        
35565    <p>Свечи зажигания прикручиваются к блоку двиг...
35566    <p>Воздушный фильтр установлен в автомобиле дл...
35567    <p>Цепь ГРМ &ndash; вариант приведения в дейст...
35568    <p>Изменение климата привело к ужесточению эко...
35569    <p>Длительность эксплуатации каталитического н...
Name: Основной текст, Length: 35570, dtype: object
15603    /acura/diagnostika-avtomobilya/zamena-toplivno...
Name: _URL_, dtype: object
Не найдена услуга Диагностика автомобиля для 
Марка                          Acura
Модель                           CSX
URL       https://ddcar.ru/acura/csx
Услуга     Замена топливного фильтра
Name: 1, dtype: obje

C:\Users\lalko\Anaconda3\envs\seo\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: `item` has been deprecated and will be removed in a future version


16139    /bmw/x1/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16143    /bmw/x3/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16147    /bmw/x4/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16150    /bmw/x5/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16154    /bmw/x6/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16158    /bmw/z4/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16162    /bmw/1m/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16166    /bmw/m3/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16170    /bmw/m4/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16174    /bmw/m5/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16178    /bmw/m6/diagnostika-avtomobilya/zamena-toplivn...
Name: _URL_, dtype: object
16182    /bmw/x5-m/diagnostika-avtomobilya/zamena-topl

ValueError: can only convert an array of size 1 to a Python scalar

In [175]:
db_all_page = pd.read_csv('export_20200213-103536.csv', encoding = 'cp1251', sep=';')
db_all_page1 = db_all_page[db_all_page['Марка'] == 'Acura']
print(db_all_page1['Модель'])
print(pd.notna(db_all_page1['Модель'][12178]))
if pd.notna(db_all_page1['Модель'][12178]):
    print(1)

21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
        ... 
12178    ZDX
12179    ZDX
12180    ZDX
12181    ZDX
12182    ZDX
Name: Модель, Length: 248, dtype: object
True
1
